deeply refered on https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/10/06/attention/

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import os
import time

SEED = 1

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [2]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True,include_lengths=True)
# SRC 데이터에는 include_lengths 인자를 넣어주게 되면서, Encoder 의 pad_pack_sequence를 원활하게끔 지원해줍니다.
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

In [3]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))
# 영어와, 독일 데이터를 다운받으면서, train, validation, test 데이터로 나눠서 가져오게 됩니다.

In [4]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)
# 최소한 2번 이상 나오는 vocab에 대해서만, numericalize 시키게 됩니다.

In [5]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
BATCH_SIZE = 1000

train_iter, test_iter = BucketIterator.splits(datasets=(train_data,test_data),
                                            batch_sizes=(BATCH_SIZE,BATCH_SIZE),  
                                            sort_key=lambda x: len(x.src), 
                                            device=device,
                                            sort_within_batch=True, 
                                            repeat=False)

# device 는 cpu 또는 gpu 를 적용하게 되고, (pytorch에서는 Variable(device=device) 와 같은 형태로, 
#인풋 데이터에 대해서 device 를 할당해줍니다.
# batch_size 를 할당해주면, 반환값에 randomly batch가 적용됩니다.(1000개씩 묶인 상태에서, 인덱스가 랜덤으로 섞인 iterator 가 됩니다.)

In [6]:
batch = next(iter(train_iter))

In [7]:
print(type(batch.src)) # torchtext 의  Field 클래스에서 include_lengths 를 True로 할당해주었기 때문에, turple 을 반환합니다.
print(batch.src[0]) # 첫 번째 인자는 input_data가 batch_size에 맞춰 들어간 상태입니다.
print(batch.src[1][:10]) # 두 번째 인자는 각 문장 별, 길이를 의미합니다.

<class 'tuple'>
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,    4,  ...,    4,    4,    4],
        [  74,  239, 1476,  ..., 1598,  140,  319],
        ...,
        [  73,  951,  717,  ...,    3,    3,    3],
        [  26,    5,    5,  ...,    1,    1,    1],
        [   3,    3,    3,  ...,    1,    1,    1]])
tensor([23, 23, 23, 23, 23, 23, 23, 23, 23, 23])


### Encoder 에 대한 논의
- 네트워크는 기존 RNN encoder-decoder 모델에 사용된 Encoder와 같은 형태입니다.
- Attention mechanism 을 사용하기 위해서는, 이번엔 hidden layer만 반환하는 것이 아닌, sequential output인 outputs 또한 반환합니다.
- hidden 은 RNN encoder-decoder와 같이 context vector와 같은 역할을 합니다.
- outputs 은 아래에 나올, Attention class의 인자로 들어가게 됩니다.

### pack_padded_sequence, pad_packed_sequence 에 대한 논의
- 이번에는 RNN 네트워크의 pytorch 의 utility function인 packing 과 padding 에 대한 함수를 사용하였습니다.
- 사용하는 이유는, RNN 네트워크의 특징 상, 배치(batch)를 돌면서 다른 길이의 데이터를 받게 되는데, (CNN의 경우 데이터의 길이는 padding으로 일치시킵니다.) 그에 따라서, 0(n^2) 이라는 연산에 대한 비효율성이 발생하게 되는데, 이러한 문제를 줄여서 연산 효율성을 추구하기 위해 진행되는 프로세스입니다.

```python
nn.utils.rnn.pack_padded_sequence(embedded, seq_length)
```
위의 코드에서 보게 되면, embedding vector와 seq_length라는 인자가 함께 들어가게 되는데, sequence length 란, 각 문장의 길이를 의미하게 되는 것으로, 위의 torchtext 에서 Field 의 인자 ` include_length = True` 로 선언하여 length 또한 함께 이터레이터를 돌면서 반환하게끔 해놓은 상태이기 때문에, 인덱싱을 통해 쉽게 넣어줄 수 있게 됩니다.

자세한 설명은 해당 페이지를 참고하시면 좋습니다. : https://gist.github.com/HarshTrivedi/f4e7293e941b17d19058f6fb90ab0fec

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, dropout):
        
        super().__init__()     
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim,padding_idx=1)
        self.num_layers = num_layers
        self.bi_rnn = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, 
                            dropout=dropout,bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.hidden_dim = hidden_dim    
        
    def forward(self, text):
        # text 는 tuple 입니다.
        input_ = text[0] # tuple 의 첫 번째 엘리먼트는 input_data가 배치로 들어온 형태입니다.
        seq_length = text[1] # tuple 의 두 번째 엘리먼트는 input_data 각각의 문장의 길이입니다.
        embedded = self.dropout(self.embedding(input_)) #[max_length, batch_size, embedding_dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, seq_length) 
        # packed_embedded 또한 tuple형태입니다. 하지만 기존의 RNN based 모델에 인자로 넣어주면 됩니다!
        outputs, hidden = self.bi_rnn(packed_embedded)
        outputs,_ = nn.utils.rnn.pad_packed_sequence(outputs)
        # hidden 은 가변 길이의 문장을 하나의 정보로 압축시키는 context vector 라고 생각하면 된다. 
        # 이러한 convext vector는 num_layer 의 갯수만큼 있고, decoder part에서 풀게 된다.
        return outputs, hidden 
    # [max_length , batch_size , hidden_layer_dim], [1 , batch_size , hidden_layer_dim] 

In [9]:
input_dim = len(SRC.vocab.itos)
output_dim = len(TRG.vocab.itos)
embedding_dim = 256
hidden_dim = 128
num_layer = 1
dropout = 0.5
batch_size = 1000
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
enc = Encoder(input_dim,embedding_dim,hidden_dim,num_layer,dropout).to(device)
batch = next(iter(train_iter))

enc(batch.src)[0].size(),enc(batch.src)[1].size()

/home/fin/anaconda3/envs/engine_3.6/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


(torch.Size([12, 1000, 256]), torch.Size([2, 1000, 128]))

<img src = 'attention_pic.png'>

### Attention 에 대한 논의
- 자세한 논의는 차후에 링크되는 블로그의 논문 요약에서 보다 자세히 다루도록 하겠습니다.
- RNN Encoder-decoder의 문제는 바로, Encoder에서 context vector 로 input_data(source data라고 주로 칭합니다.)의 information을 하나의 벡터로 압축(compress)시킨다는 것에 있습니다. 이에 따라, 한정된 정보만을 담을 수 있다는 것이 주요 단점으로 지적됩니다.
- 해당 Attention mechanism은 이러한 문제를 해결하기 위해 제시된 것으로, source data 와 target data의 각각의 sequence 에서 각각의 sentence 간의 유사성이 존재할 것이라는 아이디어에 기반합니다.
- 예로 들어서, "나는 딥러닝이 좋아" 와 "I love deep-learning" 두 문장이 존재할 경우, "좋아" 라는 단어와 "love"라는 단어 간의 유사성이 문맥적(contextual) 의미적(semantic) 유사성이 높을 것입니다. 이에 따라, `Encoder 의 t스텝의 hidden과 Decoder의 t-1 hidden 간의 유사성을 계산`하고, (위의 등식에서 score() 라는 함수로 계산됩니다.) 이를 `확률 형태로 반환`하여(Attention weights) source data 중에 어떤 단어가 target data의 특정 스텝에서 특정 단어와 `유사성이 가장 높은지를 확률 형태`로 알려주게 됩니다. 
- 기존에는, context vector를 Decoder의 initial hidden layer로 사용하면서, target data를 그대로 RNN 네트워크를 통해 연산을 진행해주었는데, Attention weights 를 Context vector에 곱해준 즉, 가중합(weighted sum)을 진행해주면서, 학습에 유의한(significant) 데이터에 "집중" 하게 만드는 것입니다.

In [10]:
class Attention(nn.Module) : 
    def __init__(self, outputs, hidden_dim,batch_size):
        
        super().__init__()        
        self.outputs = outputs # Encoder가 출력한 outputs 입니다.
#         self.hidden = hidden # Encoder가 출력한 hidden 입니다.
        self.hidden_dim = hidden_dim # hidden_dim의 dimension 입니다.
        self.score_fc1 = nn.Linear(hidden_dim*2,hidden_dim*2)
        self.score_fc2 = nn.Linear(hidden_dim*2,hidden_dim*2)        
        self.softmax_fc = nn.Linear(hidden_dim*2,1)
        self.batch_size = batch_size
        
    def forward(self,hidden) : 
        if hidden.size()[0] == 2 : hidden = hidden.view(1,self.batch_size,-1)
        outputs = self.outputs.permute(1,0,2) # [batch_size, max_length, hidden_dim]
        hidden = hidden.permute(1,0,2) # [batch_size,1, hidden_dim]
        attention_score = torch.tanh(self.score_fc2(outputs) + self.score_fc1(hidden)) # [batch_size, max_length, hidden_dim]
        attention_weights = torch.softmax(self.softmax_fc(attention_score),dim=1) # [batch_size, max_length, 1]
        context_vector = attention_weights * outputs#[batch_size, max_length, hidden_dim]
        new_context_vector = torch.sum(context_vector,dim=1)#[batch_size, hidden_dim]
        return new_context_vector

In [11]:
outputs = enc(batch.src)[0]
hidden = enc(batch.src)[1]
attention_obj = Attention(outputs, 128,1000).to(device)
attention_obj(hidden).size()

torch.Size([1000, 256])


$$p(y_i\vert y_1,...,y_{i-1},\mathbf{x})=g(y_{y-1},s_i,c_i)$$
$$\mathbf{x} = context\ vector$$

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, num_layers,\
                 dropout, attention_obj):
        
        super().__init__()        
        self.embedding = nn.Embedding(output_dim, embedding_dim,padding_idx=1)
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers        
        self.rnn = nn.GRU(embedding_dim+hidden_dim*2,hidden_dim*2, num_layers=num_layers, \
                              dropout=dropout,batch_first=True)    
        self.dropout = nn.Dropout(dropout)
        self.fully_connect = nn.Linear(hidden_dim*2,output_dim)
        self.attention_obj = attention_obj
        self.batch_size = batch_size
        
    def forward(self, inputs, hidden, enc_outputs):
        
        atten = self.attention_obj(enc_outputs,self.hidden_dim,self.batch_size)
        context_vector = atten(hidden) #[batch_size, hidden_dim]
        # 함수 내에서 Attention class을 호출해, context_vector를 계산해줍니다.
        # 이 때, attention_obj 에서 변하는 argument 는 hidden layer의 값입니다.
        embedding_vector = self.dropout(self.embedding(inputs)).permute(1,0,2) #[batch_size, max_length, embed_dim]
        #torch.Size([1000, 1, 256]) torch.Size([1000, 1, 256])
        concated_vector = torch.cat((context_vector.unsqueeze(1), embedding_vector), dim=2)
        outputs,hidden = self.rnn(concated_vector)
            
        final_outputs = outputs.squeeze(1)
        fc_layer = self.fully_connect(final_outputs)

        if len(fc_layer.size()) < 3 :
            fc_layer = fc_layer.unsqueeze(1)
            fc_layer = fc_layer.permute(1,0,2)
        else :
            fc_layer = fc_layer.permute(1,0,2)
        
        return fc_layer, hidden
#     [max_length, batch_size, output_dim] , [1, batch_size, output_dim]

In [13]:
input_dim = len(SRC.vocab.itos)
output_dim = len(TRG.vocab.itos)
embedding_dim = 256
hidden_dim = 128
num_layer = 1
dropout = 0.5
attention_obj = Attention
batch_size = 1000
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
batch = next(iter(train_iter))
dec = Decoder(output_dim,embedding_dim,hidden_dim,num_layer,dropout,attention_obj).to(device)

enc_outputs, enc_hidden  = enc(batch.src)[0],enc(batch.src)[1]
fc_layer,hidden = dec(batch.trg[0].unsqueeze(0),enc_hidden,enc_outputs)
fc_layer.size(), hidden.size()
#[max_length, batch_size, output_dim] , [1, batch_size, output_dim]

(torch.Size([1, 1000, 5893]), torch.Size([1, 1000, 256]))

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
                                
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # decoder 의 맨처음에는 encoder 에서 나온 hidden,cell을 넣어주어야 합니다! 이때, num_layer와 hidden_dim은 같아야 합니다!
        enc_output, enc_hidden = self.encoder(src)
        
        # decoder 를 돌면서, 각 단어에 대한 outputs값(벡터 형태)이 나오게 되는데, 이러한 값들을 아래의 outputs 변수에 저장해줍니다
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        # 맨 처음에는 문장의 시작을 알리는 sos(start of sentence) 토큰을 넣어주어야 합니다.
        dec_inputs = trg[0,:].unsqueeze(0)
        dec_hidden = enc_hidden
        for t in range(1, max_len):

            fc_layer, dec_hidden = self.decoder(dec_inputs,dec_hidden,enc_output)
#            output'dimension : [batch_size , output_dim], 여기서 output_dim 은 출현 가능한 모든 target lang 의 수 입니다.
            outputs[t] = fc_layer
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = fc_layer.max(1)[1] # 해당 글자의 numericalized index 를 넣어주어야 합니다.
            input_ = (trg[t] if teacher_force else top1)
            # teacher_forcing 을 할 경우, 실제 trg데이터를 다음 input으로 사용, 그렇지 않을 경우, 이전 state에서 가장 높은 
            # 값을 가진[나올 수 있는 모든 target vocab 리스트 중에서를 의미합니다. 확률값의 형태는 아니지만, 가장 개연성이 높은 단어를 의미합니다.]
        return outputs

In [15]:
#parameter settings
input_dim = len(SRC.vocab.itos)
output_dim = len(TRG.vocab.itos)
embedding_dim = 256
hidden_dim = 128
num_layer = 1
dropout = 0.5
attention_obj = Attention
batch_size = 1000
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
enc = Encoder(input_dim,embedding_dim,hidden_dim,num_layer,dropout)
dec = Decoder(output_dim,embedding_dim,hidden_dim,num_layer,dropout,attention_obj)
enc.to(device);dec.to(device)
batch = next(iter(train_iter))

In [16]:
seq = Seq2Seq(enc,dec,device)
seq(batch.src,batch.trg).size()

torch.Size([28, 1000, 5893])

### gradient clipping 에 대한 논의
- https://hskimim.github.io/Avoid_Exploding_gradient_in_Neural_Net_with_gradient_clipping/ 에 보다 자세히 기록하였습니다.
- RNN 베이스의 네트워크의 특징인 gradient exploding 을 방지해주는 방법론입니다.

In [17]:
class fit() : 
    
    def __init__(self, model, train_iter, test_iter, epoch = 5) : 
        self.optimizer = optim.Adam(model.parameters())
        # <pad> 토큰은 임베딩 벡터와, loss_function에 argument 로 들어가서, training 과정에서 제외됩니다.
        self.pad_idx = TRG.vocab.stoi['<pad>'] 
        self.criterion = nn.CrossEntropyLoss(ignore_index=self.pad_idx)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(device)
        self.train_iter = train_iter
        self.test_iter = test_iter
        self.epoch = epoch
            
    def train(self,clip):
    
        epoch_loss = 0 # loss per epoch
        self.model.train()
        
        for i, batch in enumerate(self.train_iter):
            print('batch : ',i,end='\r')
            src = batch.src
            trg = batch.trg

            self.optimizer.zero_grad()

            output = model(src, trg)        

            loss_output = output[1:].view(-1, output.shape[-1])
            loss_trg = trg[1:].view(-1)
            # sos 토큰을 제외하고, 차원을 맞춘 후에, output을 변수에 저장해줍니다.
            
            loss = self.criterion(loss_output, loss_trg)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            # gradient clipping
            self.optimizer.step()
            
            epoch_loss += loss.item()
        
        return epoch_loss / len(self.train_iter)
    

    def fit_by_iterate(self,clip) : 
        
        for epoch in range(self.epoch):
            print('epoch : ',epoch + 1)
            train_loss= self.train(clip)
            print("epoch's loss : {}".format(train_loss))

In [23]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
BATCH_SIZE = 1000
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT,Attention)

model = Seq2Seq(enc, dec, device).to(device)

In [24]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256, padding_idx=1)
    (bi_rnn): GRU(256, 512, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256, padding_idx=1)
    (rnn): GRU(1280, 1024, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5)
    (fully_connect): Linear(in_features=1024, out_features=5893, bias=True)
  )
)

In [28]:
fitting_process = fit(model,train_iter,test_iter)

In [29]:
fitting_process.fit_by_iterate(1)

epoch :  1


KeyboardInterrupt: 